In [26]:
import pandas as pd
import numpy as np
import ccxt
import time
from datetime import datetime, timedelta, timezone
import requests

days_to_fetch=30
exchange = ccxt.bitget()
# 1 minute timeframe
timeframe = '1m'
# Calculate the starting timestamp
since = exchange.parse8601((datetime.utcnow() - timedelta(days=days_to_fetch)).isoformat())

/var/folders/3r/df6gd3t56pq6t10_rqr2h7pr0000gn/T/ipykernel_99099/3539721314.py:13: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  since = exchange.parse8601((datetime.utcnow() - timedelta(days=days_to_fetch)).isoformat())


In [7]:
since

1756105513125

In [60]:
import ccxt
import pandas as pd
from datetime import datetime, timedelta
import time

# --- Configuration ---
# 데이터를 가져올 심볼 목록
SYMBOLS = ["BTC/USDT", "ETH/USDT", "BGB/USDT"]
# 최근 몇 일간의 데이터를 가져올지 설정
DAYS_TO_FETCH = 5

def create_kline_dataframe_with_ccxt(symbol, days):
    """
    ccxt를 사용하여 OHLCV와 Taker 매수 거래량을 포함하는 DataFrame을 생성합니다.
    """
    # ccxt Bitget 인스턴스 생성
    exchange = ccxt.bitget()

    print(f"\nFetching last {days} days of OHLCV data for {symbol}...")
    
    # ccxt는 타임스탬프를 밀리초 단위로 사용합니다.
    since = exchange.parse8601((datetime.utcnow() - timedelta(days=days)).isoformat())
    
    try:
        # 1일봉 데이터 가져오기
        ohlcv = exchange.fetch_ohlcv(symbol, '1d', since, limit=days)
        if not ohlcv:
            print(f"No kline data received for {symbol}.")
            return f"Could not create DataFrame for {symbol}."

        kline_df = pd.DataFrame(ohlcv, columns=['timestamp_ms', 'open', 'high', 'low', 'close', 'volume'])
        kline_df['timestamp'] = pd.to_datetime(kline_df['timestamp_ms'], unit='ms')
        
        print(f"✅ Successfully fetched OHLCV data for {symbol}.")

    except ccxt.Error as e:
        print(f"An error occurred fetching klines for {symbol}: {e}")
        return f"Could not create DataFrame for {symbol}."

    taker_volumes = []
    print(f"\nCalculating Taker Buy Volume for each day for {symbol}. This may take a moment...")

    # 각 일봉에 대해 Taker 매수 거래량 계산
    for index, row in kline_df.iterrows():
        start_ts = int(row['timestamp_ms'])
        end_ts = start_ts + timedelta(days=1).total_seconds() * 1000
        
        print(f"Processing data for {row['timestamp'].strftime('%Y-%m-%d')}...")
        
        all_trades_for_day_period = []
        fetch_since = start_ts
        
        # 해당 날짜의 모든 거래 내역을 가져오기 위한 루프
        while True:
            try:
                # API 속도 제한을 위해 약간의 딜레이 추가
                time.sleep(exchange.rateLimit / 1000)
                
                trades = exchange.fetch_trades(symbol, since=fetch_since, limit=1000)
                
                if not trades:
                    # 더 이상 가져올 거래 내역이 없으면 종료
                    break

                all_trades_for_day_period.extend(trades)
                
                last_trade_ts = trades[-1]['timestamp']
                
                # 가져온 마지막 거래가 해당 날짜의 범위를 넘어서면 더 이상 가져올 필요 없음
                if last_trade_ts >= end_ts:
                    break
                
                # 다음 fetch를 위해 마지막 거래의 타임스탬프 + 1ms로 설정
                fetch_since = last_trade_ts + 1

            except ccxt.Error as e:
                print(f"An error occurred fetching trades for {symbol}: {e}")
                break # 에러 발생 시 루프 종료
        
        # 가져온 모든 거래 내역 중에서 해당 날짜에 해당하는 Taker 매수 거래만 필터링하여 합산
        taker_buy_trades = [
            trade.get('cost', 0) for trade in all_trades_for_day_period 
            if start_ts <= trade['timestamp'] < end_ts and
               trade.get('side') == 'buy' and 
               trade.get('takerOrMaker') == 'taker' and
               trade.get('cost') is not None
        ]
        total_taker_buy_volume = sum(taker_buy_trades)
        taker_volumes.append(total_taker_buy_volume)

    # 계산된 거래량을 DataFrame에 새로운 컬럼으로 추가
    kline_df['takerBuyQuoteVol'] = taker_volumes
    
    # 최종 DataFrame 구조 정리
    final_df = kline_df[['timestamp', 'open', 'high', 'low', 'close', 'volume', 'takerBuyQuoteVol']]
    final_df = final_df.set_index('timestamp')
    
    print(f"\n✅ All data processed successfully for {symbol}!")
    return final_df

if __name__ == "__main__":
    for symbol in SYMBOLS:
        # Bitget API 심볼 형식은 'BTC/USDT'이지만, 파일이나 출력에서는 다른 형식을 사용할 수 있습니다.
        print(f"\n{'='*50}")
        print(f"Processing Symbol: {symbol}")
        print(f"{'='*50}")
        
        final_dataframe = create_kline_dataframe_with_ccxt(symbol, DAYS_TO_FETCH)
        
        print(f"\n--- Final Kline DataFrame for {symbol} ---")
        print(final_dataframe)
        print(f"\n--- End of data for {symbol} ---\n")




Processing Symbol: BTC/USDT

Fetching last 5 days of OHLCV data for BTC/USDT...


/var/folders/3r/df6gd3t56pq6t10_rqr2h7pr0000gn/T/ipykernel_99099/2116860941.py:22: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  since = exchange.parse8601((datetime.utcnow() - timedelta(days=days)).isoformat())


✅ Successfully fetched OHLCV data for BTC/USDT.

Calculating Taker Buy Volume for each day for BTC/USDT. This may take a moment...
Processing data for 2025-09-20...
Processing data for 2025-09-21...
Processing data for 2025-09-22...
Processing data for 2025-09-23...
Processing data for 2025-09-24...

✅ All data processed successfully for BTC/USDT!

--- Final Kline DataFrame for BTC/USDT ---
                 open       high        low      close        volume  \
timestamp                                                              
2025-09-20  115626.95  116124.23  115413.00  115676.33   2543.299793   
2025-09-21  115676.33  115812.57  115180.68  115244.87   2936.684552   
2025-09-22  115244.87  115379.78  111832.69  112655.18  12886.394256   
2025-09-23  112655.18  113300.00  111468.09  112009.50   5973.597286   
2025-09-24  112009.50  112919.99  111051.88  112909.10   3325.345164   

            takerBuyQuoteVol  
timestamp                     
2025-09-20                 0  
2025-09-

/var/folders/3r/df6gd3t56pq6t10_rqr2h7pr0000gn/T/ipykernel_99099/2116860941.py:22: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  since = exchange.parse8601((datetime.utcnow() - timedelta(days=days)).isoformat())


✅ Successfully fetched OHLCV data for ETH/USDT.

Calculating Taker Buy Volume for each day for ETH/USDT. This may take a moment...
Processing data for 2025-09-20...
Processing data for 2025-09-21...
Processing data for 2025-09-22...
Processing data for 2025-09-23...
Processing data for 2025-09-24...

✅ All data processed successfully for ETH/USDT!

--- Final Kline DataFrame for ETH/USDT ---
               open     high      low    close         volume  \
timestamp                                                       
2025-09-20  4469.30  4509.00  4454.49  4480.34   82100.115786   
2025-09-21  4480.34  4498.41  4443.07  4445.09   88707.135251   
2025-09-22  4445.09  4456.40  4072.91  4199.09  370204.390752   
2025-09-23  4199.09  4229.32  4114.71  4164.35  203073.885717   
2025-09-24  4164.35  4194.91  4074.54  4175.76   90247.545551   

            takerBuyQuoteVol  
timestamp                     
2025-09-20                 0  
2025-09-21                 0  
2025-09-22                

/var/folders/3r/df6gd3t56pq6t10_rqr2h7pr0000gn/T/ipykernel_99099/2116860941.py:22: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  since = exchange.parse8601((datetime.utcnow() - timedelta(days=days)).isoformat())


✅ Successfully fetched OHLCV data for BGB/USDT.

Calculating Taker Buy Volume for each day for BGB/USDT. This may take a moment...
Processing data for 2025-09-20...
Processing data for 2025-09-21...
Processing data for 2025-09-22...
Processing data for 2025-09-23...
Processing data for 2025-09-24...

✅ All data processed successfully for BGB/USDT!

--- Final Kline DataFrame for BGB/USDT ---
             open   high    low  close        volume  takerBuyQuoteVol
timestamp                                                             
2025-09-20  5.113  5.332  5.113  5.291  3.852009e+07                 0
2025-09-21  5.291  5.395  5.209  5.238  4.732723e+07                 0
2025-09-22  5.238  5.240  4.929  5.169  1.023712e+08                 0
2025-09-23  5.169  5.277  5.086  5.248  5.275705e+07                 0
2025-09-24  5.248  5.303  5.159  5.253  3.471332e+07                 0

--- End of data for BGB/USDT ---



In [59]:
final_dataframe

,open,high,low,close,volume,takerBuyQuoteVol
timestamp,,,,,,
2025-09-20,5.113,5.332,5.113,5.291,3.852009e+07,0
2025-09-21,5.291,5.395,5.209,5.238,4.732723e+07,0
2025-09-22,5.238,5.240,4.929,5.169,1.023712e+08,0
2025-09-23,5.169,5.277,5.086,5.248,5.275705e+07,0
2025-09-24,5.248,5.303,5.159,5.252,3.451580e+07,0


In [61]:
import requests
import json

url = "https://crypto.fin.cloud.ainode.ai/a71eaf04-802f-40be-93c2-5bee2548f4db/get/info/coin"
response = requests.get(url)
data = response.json()

symbols = []
for item in data['data']:
    symbols.append(item['coin_nm'] + '/USDT')

In [55]:
url = "https://api.bitget.com/api/v2/mix/market/history-candles?symbol=BTCUSDT&granularity=1D&limit=200&productType=usdt-futures"
response = requests.get(url)
data = response.json()
data

{'code': '00000',
 'msg': 'success',
 'requestTime': 1758706597523,
 'data': [['1750867200000',
   '106982.1',
   '108222.2',
   '106520',
   '107256.1',
   '49089.06493453',
   '5273004930.236655396'],
  ['1750953600000',
   '107256.1',
   '107960.7',
   '106317.5',
   '107251.8',
   '46647.99884372',
   '4992544714.78257632'],
  ['1751040000000',
   '107251.8',
   '107672.1',
   '106393.4',
   '107406.7',
   '29140.7504494',
   '3122294208.862670316'],
  ['1751126400000',
   '107406.7',
   '108480',
   '107095.8',
   '107700.8',
   '34087.32589406',
   '3674893251.205592185'],
  ['1751212800000',
   '107700.8',
   '108738.4',
   '106691.7',
   '107525.6',
   '55594.01025659',
   '5986719815.386507259'],
  ['1751299200000',
   '107525.6',
   '107763.3',
   '105578.3',
   '105910',
   '54340.80519843',
   '5795940418.157013736'],
  ['1751385600000',
   '105910',
   '108710',
   '105050',
   '108693.5',
   '78155.01383054',
   '8363760117.079407243'],
  ['1751472000000',
   '108693.5',


In [56]:
pd.DataFrame(data)

,code,msg,requestTime,data
0,00000,success,1758706597523,"[1750867200000, 106982.1, 108222.2, 106520, 10..."
1,00000,success,1758706597523,"[1750953600000, 107256.1, 107960.7, 106317.5, ..."
2,00000,success,1758706597523,"[1751040000000, 107251.8, 107672.1, 106393.4, ..."
3,00000,success,1758706597523,"[1751126400000, 107406.7, 108480, 107095.8, 10..."
4,00000,success,1758706597523,"[1751212800000, 107700.8, 108738.4, 106691.7, ..."
...,...,...,...,...
85,00000,success,1758706597523,"[1758211200000, 117526.5, 117855.5, 115423, 11..."
86,00000,success,1758706597523,"[1758297600000, 115825.9, 116084.1, 115059.3, ..."
87,00000,success,1758706597523,"[1758384000000, 115929.9, 116026.8, 115271.9, ..."
88,00000,success,1758706597523,"[1758470400000, 115471.7, 115622.9, 111549.3, ..."


In [ ]:
curl "https://api.bitget.com/api/v2/mix/market/history-candles?symbol=BTCUSDT&granularity=1W&limit=200&productType=usdt-futures"

In [33]:
symbols

['BTC/USDT',
 'USDT/USDT',
 'ETH/USDT',
 'IOST/USDT',
 'LTC/USDT',
 'BCH/USDT',
 'QTUM/USDT',
 'ETC/USDT',
 'SOLS/USDT',
 'BEBE/USDT',
 'CSAS/USDT',
 'COQ/USDT',
 'MICE/USDT',
 'COM/USDT',
 'ZERO/USDT',
 'GRAPE/USDT',
 'BTCMEME/USDT',
 'UP/USDT',
 'MMSS/USDT',
 'NFP/USDT',
 'SNEK/USDT',
 'ANALOS/USDT',
 'SILLY/USDT',
 'AIMX/USDT',
 'ARTY/USDT',
 '3ULL/USDT',
 'PEPECOIN/USDT',
 '1CAT/USDT',
 'SCPT/USDT',
 'MOBILE/USDT',
 'SIX/USDT',
 'ZKF/USDT',
 'STRK/USDT',
 'ELA/USDT',
 'BRCT/USDT',
 'FNSA/USDT',
 'OORT/USDT',
 'PROPS/USDT',
 'XPET/USDT',
 'WOOF/USDT',
 'DUSD/USDT',
 'RDNT/USDT',
 'OX/USDT',
 'GRAIL/USDT',
 'AGIX/USDT',
 'IMGNAI/USDT',
 'ORAI/USDT',
 'PSI/USDT',
 'AI/USDT',
 'ALI/USDT',
 'ARBINU/USDT',
 'SOLO/USDT',
 'APP/USDT',
 'MMIT/USDT',
 'PKEY/USDT',
 'MCH/USDT',
 'RABBIT/USDT',
 'ISLM/USDT',
 'ORDS/USDT',
 'CORE/USDT',
 'XAI/USDT',
 'ZYB/USDT',
 '$AI/USDT',
 'MDAO/USDT',
 'STRIKE/USDT',
 'DPX/USDT',
 'PLS/USDT',
 'CAI/USDT',
 'FLR/USDT',
 'BFR/USDT',
 'JONES/USDT',
 'THENA/USD

In [36]:
data = fetch_bitget_data(symbols, days_to_fetch=30)

/var/folders/3r/df6gd3t56pq6t10_rqr2h7pr0000gn/T/ipykernel_99099/2911671879.py:19: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  since = exchange.parse8601((datetime.utcnow() - timedelta(days=days_to_fetch)).isoformat())


Fetching data for BTC/USDT...
Fetching data for USDT/USDT...
Error fetching data for USDT/USDT: bitget does not have market symbol USDT/USDT
Fetching data for ETH/USDT...
Fetching data for IOST/USDT...
Fetching data for LTC/USDT...
Fetching data for BCH/USDT...
Fetching data for QTUM/USDT...
Fetching data for ETC/USDT...
Fetching data for SOLS/USDT...
Error fetching data for SOLS/USDT: bitget does not have market symbol SOLS/USDT
Fetching data for BEBE/USDT...
Error fetching data for BEBE/USDT: bitget does not have market symbol BEBE/USDT
Fetching data for CSAS/USDT...
Error fetching data for CSAS/USDT: bitget does not have market symbol CSAS/USDT
Fetching data for COQ/USDT...
Fetching data for MICE/USDT...
Error fetching data for MICE/USDT: bitget does not have market symbol MICE/USDT
Fetching data for COM/USDT...
Error fetching data for COM/USDT: bitget does not have market symbol COM/USDT
Fetching data for ZERO/USDT...
Error fetching data for ZERO/USDT: bitget does not have market s

In [38]:
data.columns

MultiIndex([(  'BTCUSDT',   'open'),
            (  'BTCUSDT',   'high'),
            (  'BTCUSDT',    'low'),
            (  'BTCUSDT',  'close'),
            (  'BTCUSDT', 'volume'),
            (  'ETHUSDT',   'open'),
            (  'ETHUSDT',   'high'),
            (  'ETHUSDT',    'low'),
            (  'ETHUSDT',  'close'),
            (  'ETHUSDT', 'volume'),
            ...
            ('TAIKOUSDT',   'open'),
            ('TAIKOUSDT',   'high'),
            ('TAIKOUSDT',    'low'),
            ('TAIKOUSDT',  'close'),
            ('TAIKOUSDT', 'volume'),
            (   'BBUSDT',   'open'),
            (   'BBUSDT',   'high'),
            (   'BBUSDT',    'low'),
            (   'BBUSDT',  'close'),
            (   'BBUSDT', 'volume')],
           length=1895)

In [39]:
1985/5

397.0

In [29]:
exchange = ccxt.bitget()
markets = exchange.load_markets()

In [30]:
available_symbols = set(markets.keys())

In [31]:
available_symbols

{'H/USDT:USDT',
 'ULTI/USDT',
 'X/USDT',
 'HIVE/USDT:USDT',
 '1000RATS/USDT:USDT',
 'BGB/ETH',
 'LMWR/USDT',
 'AI16Z/USDT',
 'APT/USDC',
 'KAITO/USDT',
 'MAV/USDT:USDT',
 'ARB/USDC:USDC',
 'CUDIS/USDT:USDT',
 'AR/USDT',
 'STX/USDT:USDT',
 'BSV/USDT:USDT',
 'NAVX/USDT',
 'BOME/USDT:USDT',
 'AXL/USDT',
 'ORCL/USDT:USDT',
 'NEWT/USDT:USDT',
 'W/USDT:USDT',
 'VINE/USDT:USDT',
 'FUSE/USDT',
 'ENA/USDT:USDT',
 'ALICE/USDT:USDT',
 'KERNEL/USDT',
 'BTC/USD:BTC',
 'CHZ/USDT:USDT',
 'IDOL/USDT',
 'MBOX/USDT',
 'NFP/USDT',
 'DOT/USD:DOT',
 'JASMY/USDT',
 'SAND/USDT:USDT',
 'SLERF/USDT:USDT',
 'ENA/EUR',
 'WOLF/USDT',
 'ALGO/USDT',
 'HIVE/USDT',
 'BCH/USDC',
 'CETUS/USDT',
 'BCH/USDT:USDT',
 'SOL/USDT',
 'BTC/USDC',
 'CESS/USDT',
 '1000XEC/USDT:USDT',
 'KILO/USDT',
 'AAVE/USDT:USDT',
 'AIOZ/USDT:USDT',
 'APT/USD:APT',
 'RPL/USDT',
 'STX/USDC:USDC',
 'EGLD/USDT:USDT',
 'PRCL/USDT',
 'HOME/USDT:USDT',
 'FET/USDT:USDT',
 'AVAX/USDT',
 'GHST/USDT:USDT',
 'BAT/USDT:USDT',
 'XCX/USDT:USDT',
 'WLD/USDC:U

In [49]:
import pandas as pd
import numpy as np
import ccxt
import time
from datetime import datetime, timedelta, timezone
import requests

def strategy(df, config_dict):
    """
    Implements an intraday trading strategy that weights momentum by volume and trade imbalance.
    
    NOTE: This function is self-contained. It fetches its own data from external APIs
    and IGNORES the input DataFrame `df`. This is to comply with server environments
    where only this single function can be executed.
    """
    # --- Input validation for config_dict ---
    if not isinstance(config_dict, dict):
        raise TypeError("config_dict must be a dictionary.")

    # --- Part 1: Fetch Tradable Symbols ---
    symbols_url = "https://crypto.fin.cloud.ainode.ai/a71eaf04-802f-40be-93c2-5bee2548f4db/get/info/coin"
    print("Fetching tradable symbols...")
    try:
        response = requests.get(symbols_url)
        response.raise_for_status()
        coins_data = response.json()
        
        symbols_to_fetch = []
        for item in coins_data['data']:
            coin_name = item.get('coin_nm')
            if coin_name and coin_name.upper() != 'USDT':
                symbols_to_fetch.append(f"{coin_name}/USDT")
        
        print(f"Successfully found {len(symbols_to_fetch)} tradable symbols.")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching symbols: {e}")
        print("Using default symbols as a fallback.")
        symbols_to_fetch = ['BTC/USDT', 'ETH/USDT']

    if not symbols_to_fetch:
        print("No symbols to process. Exiting.")
        return {}

    # --- Part 2: Fetch OHLCV Data from Bitget ---
    # Fetching 1 day of 1-minute data is sufficient for a 20-minute rolling window.
    days_to_fetch = 1 
    exchange = ccxt.bitget()
    
    print("Loading available markets from Bitget...")
    try:
        markets = exchange.load_markets()
        available_symbols = set(markets.keys())
        print(f"Found {len(available_symbols)} markets on Bitget.")
    except Exception as e:
        print(f"Could not load markets from Bitget: {e}. Proceeding without market validation.")
        available_symbols = set(symbols_to_fetch)
    
    timeframe = '1m'
    since = exchange.parse8601((datetime.now(timezone.utc) - timedelta(days=days_to_fetch)).isoformat())
    
    all_symbols_data = []
    
    for symbol in symbols_to_fetch:
        if symbol not in available_symbols:
            continue

        print(f"Fetching data for {symbol}...")
        try:
            limit = 1000 # Fetch up to 1000 minutes (about 16.7 hours)
            ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit=limit)
            
            if not ohlcv:
                print(f"Warning: No data returned for {symbol}.")
                continue

            fetched_df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
            fetched_df['timestamp'] = pd.to_datetime(fetched_df['timestamp'], unit='ms')
            fetched_df.set_index('timestamp', inplace=True)
            
            # --- CRITICAL DATA NOTE ---
            # The strategy requires 'takerBuyBaseTvol' which is not available in standard OHLCV data.
            # In a real-world scenario, you MUST replace this with actual data.
            # Here, we use a neutral placeholder (50% of total volume) so the logic can run.
            fetched_df['takerBuyBaseTvol'] = fetched_df['volume'] * 0.5

            fetched_df.columns = pd.MultiIndex.from_product([[symbol.replace('/', '')], fetched_df.columns])
            all_symbols_data.append(fetched_df)

        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")
        
        time.sleep(exchange.rateLimit / 1000)

    if not all_symbols_data:
        print("Could not fetch any data. Strategy cannot proceed.")
        return {}
        
    data_df = pd.concat(all_symbols_data, axis=1)

    # --- Part 3: Strategy Logic (Intraday Momentum-Weighted by Volume Imbalance) ---
    
    strategy_specific_config = config_dict.get("strategy_config", {})
    rolling_window = strategy_specific_config.get("rolling_window", 20)
    epsilon = 1e-9 # Small constant to avoid division by zero

    symbols_in_df = data_df.columns.get_level_values(0).unique()
    final_signals = {}

    for symbol in symbols_in_df:
        symbol_df = data_df[symbol].copy() # Use .copy() to avoid SettingWithCopyWarning
        
        # Ensure there's enough data for the rolling window
        if len(symbol_df) < rolling_window + 1:
            continue

        # 1. Minute return
        # Using .loc to prevent potential SettingWithCopyWarning
        symbol_df.loc[:, 'r_min'] = (symbol_df['close'] - symbol_df['open']) / (symbol_df['open'] + epsilon)

        # 2. Return statistics (rolling over the last 20 minutes, excluding the current minute)
        shifted_returns = symbol_df['r_min'].shift(1)
        mu_20 = shifted_returns.rolling(window=rolling_window).mean()
        sigma_20 = shifted_returns.rolling(window=rolling_window).std()

        # 3. Return z-score
        z_ret = (symbol_df['r_min'] - mu_20) / (sigma_20 + epsilon)

        # 4. Volume statistics (same 20-minute window)
        shifted_volume = symbol_df['volume'].shift(1)
        mu_vol = shifted_volume.rolling(window=rolling_window).mean()
        sigma_vol = shifted_volume.rolling(window=rolling_window).std()
        
        # 5. Volume z-score
        z_vol = (symbol_df['volume'] - mu_vol) / (sigma_vol + epsilon)

        # 6. Volume imbalance
        tvol = symbol_df['volume']
        taker_buy_vol = symbol_df['takerBuyBaseTvol']
        imb = (2 * taker_buy_vol - tvol) / (tvol + epsilon)
        
        # 7. Final signal
        signal = z_ret * z_vol * imb
        
        # Store the latest valid signal for the symbol
        if not np.isnan(signal.iloc[-1]):
            final_signals[symbol] = signal.iloc[-1]

    if not final_signals:
        print("No valid signals generated.")
        return {}

    # --- Cross-Sectional Normalization & Weight Allocation ---
    final_signals_series = pd.Series(final_signals)
    
    # Optional: Neutralize by cross-sectional z-score (can be added if desired)
    # cs_mean = final_signals_series.mean()
    # cs_std = final_signals_series.std()
    # if cs_std > 0:
    #     final_signals_series = (final_signals_series - cs_mean) / cs_std

    total_abs_signal = final_signals_series.abs().sum()
    if total_abs_signal == 0:
        return {}
        
    weights = final_signals_series / total_abs_signal
    
    print("\nCalculated Strategy Weights:")
    sorted_weights = sorted(weights.items(), key=lambda item: abs(item[1]), reverse=True)
    for symbol, weight in sorted_weights:
        print(f"  {symbol}: {weight:.6f}")
            
    return weights.to_dict()



In [ ]:
config_params = {"strategy_config": {
    # 이동평균 및 표준편차 계산을 위한 롤링 윈도우 크기 (분 단위)
    # 전략의 핵심 파라미터입니다.
    "rolling_window": 20
  }}

In [47]:
config_params.get("strategy_config", {})

{'rolling_window': 20}

In [41]:
# 이 파일은 strategy.py에서 사용할 설정값을 정의합니다.
strategy_config_params = {
  "rebalancing_config": {
    # 리밸런싱 주기는 외부 스케줄러에서 사용될 수 있으나,
    # 현재 전략은 실행될 때마다 최신 분봉 데이터로 신호를 계산합니다.
    "rebalancing_interval_hours": 72,
  },
  "strategy_config": {
    # 이동평균 및 표준편차 계산을 위한 롤링 윈도우 크기 (분 단위)
    # 전략의 핵심 파라미터입니다.
    "rolling_window": 20
  }
}


In [ ]:
import pandas as pd
import numpy as np
import ccxt
import time
from datetime import datetime, timedelta, timezone

def strategy(df, config_dict):
    """
    Implements an intraday trading strategy that weights momentum by volume and trade imbalance.
    
    NOTE: This function is self-contained. It fetches its own data directly from the exchange
    and IGNORES the input DataFrame `df`.
    """
    # --- Input validation for config_dict ---
    if not isinstance(config_dict, dict):
        raise TypeError("config_dict must be a dictionary.")

    # --- Part 1: Fetch Tradable Symbols Directly from Bitget ---
    exchange = ccxt.bitget()
    print("Loading available markets from Bitget...")
    try:
        markets = exchange.load_markets()
        # Filter for active USDT spot markets
        symbols_to_fetch = [
            m['symbol'] for m in markets.values()
            if m.get('spot') and m.get('active') and m['symbol'].endswith('/USDT')
        ]
        
        if not symbols_to_fetch:
            raise ValueError("No active USDT spot markets found on Bitget.")
        
        # Limit the number of symbols to avoid excessive API calls in a single run
        # For a real-world scenario, you might want to filter by volume or other criteria
        max_symbols = 50 
        symbols_to_fetch = symbols_to_fetch[:max_symbols]
        print(f"Found {len(symbols_to_fetch)} active USDT spot markets to process.")

    except Exception as e:
        print(f"Error fetching symbols from Bitget: {e}")
        print("Using a small default list as a fallback.")
        symbols_to_fetch = ['BTC/USDT', 'ETH/USDT', 'BGB/USDT']

    # --- Part 2: Fetch OHLCV Data from Bitget ---
    days_to_fetch = 1 
    timeframe = '1m'
    since = exchange.parse8601((datetime.now(timezone.utc) - timedelta(days=days_to_fetch)).isoformat())
    
    all_symbols_data = []
    
    for symbol in symbols_to_fetch:
        print(f"Fetching data for {symbol}...")
        try:
            limit = 1000 # Fetch up to 1000 minutes
            ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit=limit)
            
            if not ohlcv:
                print(f"Warning: No data returned for {symbol}.")
                continue

            fetched_df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
            fetched_df['timestamp'] = pd.to_datetime(fetched_df['timestamp'], unit='ms')
            fetched_df.set_index('timestamp', inplace=True)
            
            # --- CRITICAL DATA NOTE ---
            # The strategy requires 'takerBuyBaseTvol' which is not available in standard OHLCV data.
            # In a real-world scenario, you MUST replace this with actual data.
            # Here, we use a neutral placeholder (50% of total volume) so the logic can run.
            fetched_df['takerBuyBaseTvol'] = fetched_df['volume'] * 0.5

            fetched_df.columns = pd.MultiIndex.from_product([[symbol.replace('/', '')], fetched_df.columns])
            all_symbols_data.append(fetched_df)

        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")
        
        time.sleep(exchange.rateLimit / 1000)

    if not all_symbols_data:
        print("Could not fetch any data. Strategy cannot proceed.")
        return {}
        
    data_df = pd.concat(all_symbols_data, axis=1)

    # --- Part 3: Strategy Logic (Intraday Momentum-Weighted by Volume Imbalance) ---
    strategy_specific_config = config_dict.get("strategy_config", {})
    rolling_window = strategy_specific_config.get("rolling_window", 20)
    epsilon = 1e-9

    symbols_in_df = data_df.columns.get_level_values(0).unique()
    final_signals = {}

    for symbol in symbols_in_df:
        symbol_df = data_df[symbol].copy()
        
        if len(symbol_df) < rolling_window + 1:
            continue

        symbol_df.loc[:, 'r_min'] = (symbol_df['close'] - symbol_df['open']) / (symbol_df['open'] + epsilon)

        shifted_returns = symbol_df['r_min'].shift(1)
        mu_20 = shifted_returns.rolling(window=rolling_window).mean()
        sigma_20 = shifted_returns.rolling(window=rolling_window).std()
        z_ret = (symbol_df['r_min'] - mu_20) / (sigma_20 + epsilon)

        shifted_volume = symbol_df['volume'].shift(1)
        mu_vol = shifted_volume.rolling(window=rolling_window).mean()
        sigma_vol = shifted_volume.rolling(window=rolling_window).std()
        z_vol = (symbol_df['volume'] - mu_vol) / (sigma_vol + epsilon)

        tvol = symbol_df['volume']
        taker_buy_vol = symbol_df['takerBuyBaseTvol']
        imb = (2 * taker_buy_vol - tvol) / (tvol + epsilon)
        
        signal = z_ret * z_vol * imb
        
        if not np.isnan(signal.iloc[-1]):
            final_signals[symbol] = signal.iloc[-1]

    if not final_signals:
        print("No valid signals generated.")
        return {}

    # --- Weight Allocation ---
    final_signals_series = pd.Series(final_signals)
    total_abs_signal = final_signals_series.abs().sum()
    
    if total_abs_signal == 0:
        return {}
        
    weights = final_signals_series / total_abs_signal
    
    print("\nCalculated Strategy Weights:")
    sorted_weights = sorted(weights.items(), key=lambda item: abs(item[1]), reverse=True)
    for symbol, weight in sorted_weights:
        # Print only non-zero weights
        if abs(weight) > 0:
            print(f"  {symbol}: {weight:.6f}")
            
    return weights.to_dict()



In [50]:
strategy(pd.DataFrame(), config_params)

Fetching tradable symbols...
Successfully found 1109 tradable symbols.
Loading available markets from Bitget...
Found 1457 markets on Bitget.
Fetching data for BTC/USDT...
Fetching data for ETH/USDT...
Fetching data for IOST/USDT...
Fetching data for LTC/USDT...
Fetching data for BCH/USDT...
Fetching data for QTUM/USDT...
Fetching data for ETC/USDT...
Fetching data for COQ/USDT...
Fetching data for NFP/USDT...
Fetching data for SNEK/USDT...
Fetching data for STRK/USDT...
Fetching data for ELA/USDT...
Fetching data for OORT/USDT...
Fetching data for PROPS/USDT...
Fetching data for RDNT/USDT...
Fetching data for ORAI/USDT...
Fetching data for AI/USDT...
Fetching data for SOLO/USDT...
Fetching data for MCH/USDT...
Fetching data for CORE/USDT...
Fetching data for XAI/USDT...
Fetching data for UPC/USDT...
Fetching data for BLUR/USDT...
Fetching data for CHESS/USDT...
Fetching data for RARE/USDT...
Fetching data for AVA/USDT...
Fetching data for ALT/USDT...
Fetching data for RON/USDT...
Fetc

{}